# Исследование надёжности заёмщиков

## Описание проекта

**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель исследования** - результаты будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# загрузка необходимых библиотек для проведения исследования
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
clients = pd.read_csv('/datasets/data.csv') # откурытие файла с данными и запись его в переменную `clients`
clients.info() # вывод общей информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**По общей информации можно сказать:**
1. В таблице присутствует 3 типа данных: `int`(целые числа), `float`(дробные) и `object`(может содержать несколько типов данных).
2. В столбцах `days_employed` и `total_income` отсутствует часть значений.
3. В названии столбцов нарушений нет, можно их не переименовывать.

In [3]:
clients.isna().sum() # проверим сколько пропущенных значений, применив метод .isna() и просуммировав значения методом .sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропущенных значений по столбцам `days_employed` и `total_income` совпадает. 10% пропущенных значений могут повлиять на исследование, их нужно будет обработать.

In [4]:
clients.head(10) # выведем 10 первых строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**По 10 первым строкам таблицы можно отметить следующее:**
- В таблице присутствуют отрицательные значения в столбце `days_employed`. Кроме этого, нереальное значение по строке 4 - 932 года стаж. Т.к.информация по стажу в исследовании мы применять не будем, оставим большие значения без изменений.
- В столбце `education` разный регистр записи, что не позволит в дальнейшем группировать данные.
- В столбце `purpose` есть похожие значения, но записанные по-разному, требуется приведение их к одной категирии (пример, "покупка жилья" и "покупка жилья для семьи").

In [5]:
clients.tail(10) # выведем 10 последних строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


По последним 10 строкам выводы аналогичные.

Далее посмотрим в каких категориях присутствуют пропущенные значения:

In [6]:
print(clients[(clients['days_employed'].isna()==True) & (clients['total_income'].isna()==True)]['income_type'].value_counts())
print(clients[(clients['days_employed'].isna()==True) & (clients['total_income'].isna()==True)]['income_type'].value_counts().sum())


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
2174


1. Общее количество пропусков - 2174.
2. Пропуски могли возникнуть из-за человеческого фактора, при внесении данных не все строки были заполнены. Или из-за не именеем информации ячейки были оставлены пустыми.
3. Т.к. пропущенные значения находятся в разных категориях, то проавильнее их будет заполнить, взяв среднее значение по группам типа занятости.

In [7]:
clients.describe() # для дополнительного анализа данных выведем статистическую таблицу методом .describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**По статистическим данным можно отметить следующее:**
- Столбец `children` - появилось отрицательное значение (-1).
- Столбец `days_employed` - присутствуют ошибки, которые были обнаружены выше (нереальные значения при mean - 173 года и max - 1100 лет).
- Столбец `dob_years` - есть нулевые значения, возможно, значение не указано при заполнении.

Посчитаем количество некорректных значений:

In [8]:
print('Количество детей с отрицательным значением:', clients[clients['children'] < 0]['children'].count())
print('Количество строк с отрицательным стажем:', clients[clients['days_employed'] < 0]['days_employed'].count())
print('Количество строк с завышенным стажем:', clients[clients['days_employed'] > 21535]['days_employed'].count())
# завышенный стаж сравнивается со значением = 75 (макс.возраст таблицы) - 16 (офиц.возраст начала работы) * 365 дней в году
print('Количество строк с нулевым возрастом:', clients[clients['dob_years'] == 0]['dob_years'].count())



Количество детей с отрицательным значением: 47
Количество строк с отрицательным стажем: 15906
Количество строк с завышенным стажем: 3445
Количество строк с нулевым возрастом: 101


**Выводы**
1. В таблице присутствуют пропущенные значения в количестве 2174 (в столбцах `days_employed` и `total_income`), которые нужно будет заполнить. Тип данных также в этих столбцах можно заментить на `int` для удобства дальнейшего анализа.
2. Требуется обработка отрицательных значений в столбцах `days_employed` и `children`.
3. Столбец `education` следует привести к нижнему регистру данных.
4. По таблице требуется проверка и обработка дубликатов.
5. Значения столбца `purpose` требуется лемматезировать.
6. Т.к.возраст заемщика не будет использован в анализе, не будем менять данные в столбце `dob_years`.

## Шаг 2. Предобработка данных

### Обработка пропусков

Уберем отрицательные значения в таблице:

In [9]:
clients['children'] = abs(clients['children']) # возвращаем значения столбца 'children' по модулю, используя abs()
clients['days_employed'] = abs(clients['days_employed']) # возвращаем значения столбца 'days_employed' по модулю
# проверяем результат на таблице со статистическими данными
clients.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Отрицательные значения устранены.

Далее заполним пропуски в данных средними значениями по каждой группе.

In [10]:
# расчет среднего значения стажа по типам занятости с применением метода .groupby для групировки и метода .mean() для определения среднего значения
days_employed_median = clients.groupby('income_type')['days_employed'].median()
print(days_employed_median)
# расчет медианы дохода по типам занятости
total_income_median = clients.groupby('income_type')['total_income'].median()
print(total_income_median)


income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64
income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [11]:
# заполняем пропуски данных медианой по группе и выводим общую информацию о данных
clients['days_employed'] = clients.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.median()))
clients['total_income'] = clients.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
clients.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:** пропущенные значения в таблице отсутствуют

### Замена типа данных

Заменим тип данных в столбцах `days_employed` и `total_income` на целочисленные (тип `int`), применив метод `.astype()`:

In [12]:
clients['days_employed'] = clients['days_employed'].astype('int') 
clients['total_income'] = clients['total_income'].astype('int')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод:**
Произвели замену типа данных в столбцах days_employed и total_income с `float` на целые значения - `int` для удобства дальнейшего анализа.

### Обработка дубликатов

Приведем значения столбца `education` к нижнему регистру, используя метод `.lower()` и выведем 10 первых строк таблицы:

In [13]:
clients['education'] = clients['education'].str.lower()
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Проверим наличие в данных дубликатов методом `.duplicated()` и посчитаем их количество методом `.sum()`

In [14]:
# проверим наличие дубликатов
print('Количество дубликатов:', clients.duplicated().sum())


Количество дубликатов: 71


В данных присутствует 71 дубликат записи. Рассмотрим их, отсортировав методом `.sort_values` по трем столбцам: 'days_employed', 'total_income', 'purpose'

In [15]:
clients[clients.duplicated(keep = False)].sort_values(by=['days_employed', 'total_income', 'purpose'])


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9374,0,1547,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
9920,0,1547,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
15991,0,1547,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
19387,0,1547,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
2254,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
5557,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
6674,0,365213,64,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
13035,0,365213,65,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
17338,0,365213,64,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу


В таблице присутствуют явные дубликаты, которые следует удалить.

In [16]:
clients = clients.drop_duplicates() # удаление дубликатов и перезапись данных
print('Количество дубликатов после удаления:', clients.duplicated().sum()) # подсчет дубликатов в данных

Количество дубликатов после удаления: 0


**Вывод**
- в данных обработаны и удалены дубликаты
- данные столбца `education` приведены к нижнему регистру

### Лемматизация

Как отмечалось ранее, данные в столбце `purpose` имеют схожие категории, но записаны по-разному, необходимо их обработать и привести к единообразию. Для этого можно применить процесс - **Лемматизации**

Посчитаем количество записей по группам в столбце `purpose`:

In [17]:
purpose_grouped = clients.groupby('purpose')['purpose'].count()
display(purpose_grouped)

purpose
автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                606
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерческой недви

Разобъем цели на "леммы" и определим их частоту методом Counter:

In [18]:
m = Mystem()
lemmas_list = [] # создаем новый список
for element in clients['purpose']:
    lemmas = m.lemmatize(element)
    lemmas_list.extend(lemmas) # дополняем список элементами из объекта lemmas
print(Counter(lemmas_list)) # определяем частоту встречаемости каждой леммы

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


По выведенным данным можно определить следующие цели кредита: "жилье", "недвижимость", "автомобиль", "свадьба" и "образование".

Сохраним цели кредита в новом списке `categories`

In [19]:
categories = ['жилье', 'недвижимость', 'автомобиль', 'свадьба', 'образование']

Лемматизируем данные и добавим новые цели в столбец `purpose_group`:

In [20]:
# запишем функцию для лемматизации
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma
# создадим новый столбец, куда запишем "леммы"
clients['purpose_group'] = clients['purpose'].apply(lemmatize)        
# выведем первые 15 строк таблицы
clients.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


из вывода видим, что "жилье" также может относиться к "недвижимости". Заменим цель "жилье" на "недвижимость" для одинакового трактования цели. В цели "недвижимость" будет находиться как коммерческая, так и жилая недвижимость.

In [21]:
# заменим значение 'жилье' на 'недвижимость' в столбце 'purpose_group', применив метод .loc[]
clients.loc[clients['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'

In [22]:
clients['purpose_group'].value_counts() # посчитаем количество записей в каждой категории

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

**Вывод**
Для анализа данные столбца `purpose` были лемматизированы в 4 категории в столбце `purpose_group`. Теперь с ними можно работать.

### Категоризация данных

Разобъем данные в столбцах на группы:

In [23]:
# запишем функцию для катигоризации данных по наличию детей
def having_children(row):
    if row['children'] == 0:
        return 0
    return 1
clients['having_children'] = clients.apply(having_children, axis=1)
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,having_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0


Для категоризации данных по уровню дохода проанализируем данные:

In [24]:
clients.groupby('income_type')['total_income'].mean() # найдем среднее значение дохода в каждой категории

income_type
безработный        131339.000000
в декрете           53829.000000
госслужащий        168862.543583
компаньон          199451.225089
пенсионер          135250.604597
предприниматель    499163.000000
сотрудник          159566.262901
студент             98201.000000
Name: total_income, dtype: float64

In [25]:
clients.groupby('income_type')['total_income'].max() # найдем максимальные значения дохода в каждой категории

income_type
безработный         202722
в декрете            53829
госслужащий         910451
компаньон          2265604
пенсионер           735103
предприниматель     499163
сотрудник          1726276
студент              98201
Name: total_income, dtype: int64

In [26]:
clients.groupby('income_type')['total_income'].min() # найдем минимальные значения дохода в каждой категории

income_type
безработный         59956
в декрете           53829
госслужащий         29200
компаньон           28702
пенсионер           20667
предприниматель    499163
сотрудник           21367
студент             98201
Name: total_income, dtype: int64

2. Катигоризируем данные по уровню дохода:
    - "низкий доход" - менее 50000 (исходя из минимального среднего по группам)
    - "средний доход" - от 50000 до 200000 (диапазон определен исходя из данных по среднему уровню)
    - "выше среднего" - от 200000 до 1000000
    - "миллионер" - более 1000000 (присутствуют в категориях данных)

In [27]:
# запишем функцию для катигоризации данных по уровню дохода
def salary_amount(row):
    if row['total_income'] <= 50000:
        return "низкий доход"
    if row['total_income'] >= 1000000:
        return "миллионер"
    if row['total_income'] >= 50000 and row['total_income'] <= 200000:
        return "средний доход"
    return "выше среднего"
clients['salary_amount'] = clients.apply(salary_amount, axis=1)
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,having_children,salary_amount
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1,выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1,выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,средний доход


**Вывод**
для ответов на вопросы мы катекогизировали следующие данные заемщикам:
- данные по наличию детей
- данные по уровню дохода
- данные о семейном положении уже были категоризированы

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопрос построим сводную таблицу, применив `.pivot_table()`:

In [28]:
# составим сводную таблицу и посчитаем количество людей по каждой группе
data_pivot = clients.pivot_table(index = ['having_children'], columns = 'debt', values = 'gender', aggfunc = 'count') 
# посчитаем вероятность задолженности и применим метод .round() для округления до 1 знака
data_pivot['loan_default'] = (data_pivot[1] / (data_pivot[0] + data_pivot[1])*100).round(1)
data_pivot.head()


debt,0,1,loan_default
having_children,,,
0,13028,1063,7.5
1,6685,678,9.2


**Вывод**
- вероятность задолженности при наличии детей выше (9,2%), чем при их отсутствии (7,5%)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим аналогичную таблицу

In [29]:
data_pivot = clients.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot['loan_default'] = (data_pivot[1] / (data_pivot[0] + data_pivot[1])*100).round(1)
data_pivot.sort_values('loan_default', ascending = False) # отсортируем значения вероятности задолженности по убыванию


debt,0,1,loan_default
family_status,,,
Не женат / не замужем,2536,274,9.8
гражданский брак,3763,388,9.3
женат / замужем,11408,931,7.5
в разводе,1110,85,7.1
вдовец / вдова,896,63,6.6


**Вывод**
- наиболее высокая вероятность появления задолженности у категорий людей не состоящих в браке или проживающих в гражданском браке
- заемщики, состоящие в браке, разведенные или вдовцы имеют вероятность намного ниже.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
data_pivot = clients.pivot_table(index = ['salary_amount'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot['loan_default'] = (data_pivot[1] / (data_pivot[0] + data_pivot[1])*100).round(1)
data_pivot.sort_values('loan_default', ascending = False)


debt,0,1,loan_default
salary_amount,,,
средний доход,14655,1360,8.5
миллионер,23,2,8.0
выше среднего,4686,356,7.1
низкий доход,349,23,6.2


**Вывод**
- наибольший процент по просрочке задолженности у людей со среднем достатком от 50 до 200 т.руб - самая массовая категория
- наименьший процент - у людей с доходом менее 50 т.руб, скорее всего в виду ограниченности финансов люди наиболее осознанно подходят к взятию кредита и его возврата.

- Как разные цели кредита влияют на его возврат в срок?

In [31]:
data_pivot = clients.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot['loan_default'] = (data_pivot[1] / (data_pivot[0] + data_pivot[1])*100).round(1)
data_pivot.sort_values('loan_default', ascending = False)


debt,0,1,loan_default
purpose_group,,,
автомобиль,3903,403,9.4
образование,3643,370,9.2
свадьба,2138,186,8.0
недвижимость,10029,782,7.2


**Вывод**
- наиболее ответственные заемщики, которые берут кредит на недвижимость (7,2% просрочек)
- чуть менее ответственны те, кто берут кредит на свадьбу (8%)
- и наименее ответственные из выборки - кто берут кредит на покупку автомобиля, либо на образование

## Шаг 4. Общий вывод



Проведя обработку и анализ представленных данных, можно выделить следующие типы заемщиков:
1. **ответственный заемщик** - человек, не имеющий детей, состоящий в официальном браке, либо ранее состоявший в браке, с уровнем дохода выше менее 50 тыс.руб, либо более 200 тыс.руб, берущие кредит на покупку недвижимости
2. **менее ответственный заемщик** - человек с детьми, не состоящий никогда в официальном браке, с уровнем дохода от 50 до 200 тыс.руб и открывающий кредит на приобретение автомобиля или образование.
- Что говорит о том, что люди в официальном браке и не имеющие иждевенцев имеют больше финансовых возможностей для обслуживания своих кредитов. 